# Verify the ProprioTunedCNN sensorprocessing

Reload a pre-trained model specified by an experiment trained by the Train-ProprioTuned-CNN notebook.

This notebook runs a number of experiments that illustrate the performance. 

In [1]:
import sys
sys.path.append("..")
from settings import Config

import pathlib
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torchvision import models, transforms
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset

from behavior_cloning.demo_to_trainingdata import BCDemonstration
from sensorprocessing.sp_propriotuned_cnn import VGG19ProprioTunedRegression, VGG19ProprioTunedSensorProcessing, ResNetProprioTunedRegression, ResNetProprioTunedSensorProcessing
from robot.al5d_position_controller import RobotPosition

# Move data to GPU (if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [2]:
# The experiment/run we are going to run: the specified model will be created
experiment = "sensorprocessing_propriotuned_cnn"
run = "vgg19_128"
# run = "resnet50_128"
exp = Config().get_experiment(experiment, run)

Loading pointer config file: C:\Users\lboloni\.config\BerryPicker\mainsettings.yaml
Loading machine-specific config file: G:\My Drive\LotziStudy\Code\PackageTracking\BerryPicker\settings\settings-LotziYoga.yaml
No system dependent experiment file
 G:\My Drive\LotziStudy\Code\PackageTracking\BerryPicker\settings\experiment-config\LotziYoga\sensorprocessing_propriotuned_cnn\vgg19_128_sysdep.yaml,
 that is ok, proceeding.
Configuration for experiment: sensorprocessing_propriotuned_cnn/vgg19_128 successfully loaded


### Check the model for an encoding mode

Check whether we can load back the model

In [3]:
if exp['model'] == 'VGG19ProprioTunedRegression':
    hidden_size = exp["latent_size"]
    output_size = Config()["robot"]["action_space_size"]
    model = VGG19ProprioTunedRegression(hidden_size, output_size)
    if exp['loss'] == 'MSELoss':        
        criterion = nn.MSELoss()
    elif exp['loss'] == 'L1Loss':
        criterion = nn.L1Loss()
elif exp['model'] == 'ResNetProprioTunedRegression':
    model = ResNetProprioTunedRegression(exp, device)
    if exp['loss'] == 'MSELoss':        
        criterion = nn.MSELoss()
    elif exp['loss'] == 'L1Loss':
        criterion = nn.L1Loss()
else:
    raise Exception(f"Unknown model {exp['model']}")

modelfile = pathlib.Path(exp["data_dir"], 
                         exp["proprioception_mlp_model_file"])
print(modelfile)
assert modelfile.exists()
model.load_state_dict(torch.load(modelfile))

c:\Users\lboloni\Documents\Code\_VirtualEnvironments\Robot\Robot-venv\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\lboloni\Documents\Code\_VirtualEnvironments\Robot\Robot-venv\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=VGG19_Weights.IMAGENET1K_V1`. You can also use `weights=VGG19_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


c:\Users\lboloni\Documents\Code\_TempData\BerryPicker-experiments\sensorprocessing_propriotuned_cnn\vgg19_128\proprioception_mlp.pth


C:\Users\lboloni\AppData\Local\Temp\ipykernel_54712\182585803.py:22: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(modelfile))


<All keys matched successfully>

In [4]:
task = exp["proprioception_testing_task"]
demos_dir = pathlib.Path(Config()["demos"]["directory"])
task_dir = pathlib.Path(demos_dir, "demos", task)
enc = model.to(device)

inputlist = []
targetlist = []
with torch.no_grad():
    for demo_dir in task_dir.iterdir():
        if not demo_dir.is_dir():
            pass
        bcd = BCDemonstration(demo_dir, sensorprocessor=None)
        for i in range(bcd.trim_from, bcd.trim_to):
            sensor_readings, _ = bcd.get_image(i)
            z = enc.encode(sensor_readings)
            break

print(z)
print(z.shape)


Cameras found: ['dev2']
There are 437 steps in this demonstration
This demonstration was recorded by the following cameras: ['dev2']
tensor([[-2.9732, -1.9004, -4.9308,  2.0603, -1.4342, -1.7257, -0.1114,  2.1723,
         -4.7429, -5.8317, -1.4768, -6.2447, -1.5482,  3.8438, -6.5623, -5.2360,
          1.5520,  1.9094,  3.0570, -4.4211, -0.9991, -2.9235, -0.8717, -0.4044,
         -3.8335, -2.1707, -2.4310,  2.9685, -1.4171,  3.7674, -0.7322, -3.4720,
         -0.7621,  5.4394, -4.5618,  1.3335, -3.2449, -2.4954, -0.2929, -0.6073,
         -0.1083, -2.6737, -4.5848, -4.1543,  0.4905, -0.6414, -1.7205, -3.0285,
         -2.5272, -2.2324, -0.0562, -3.1220, -4.1446,  1.3693,  3.8048, -2.0927,
          1.9655,  0.6850,  0.5560,  1.6852, -0.1277, -1.6638,  1.7602, -2.5111,
         -2.0892,  0.2119,  4.3101,  3.6563, -3.9038,  1.0110,  1.6067, -0.7569,
          1.9169,  4.7758, -2.7652, -1.8752,  3.2790, -1.9834,  0.3034, -2.3938,
         -6.9657, -4.4372, -1.4296, -2.3853, -4.3761, -3.

### Show how to use an SensorProcessing object

FIXME: this is VGG19 dependent

In [5]:
# Create the sensor processing package

exp = Config().get_experiment("sensorprocessing_propriotuned_cnn", run)
sp = VGG19ProprioTunedSensorProcessing(exp, device=device)

No system dependent experiment file
 G:\My Drive\LotziStudy\Code\PackageTracking\BerryPicker\settings\experiment-config\LotziYoga\sensorprocessing_propriotuned_cnn\vgg19_128_sysdep.yaml,
 that is ok, proceeding.
Configuration for experiment: sensorprocessing_propriotuned_cnn/vgg19_128 successfully loaded


c:\Users\lboloni\Documents\Code\_Checkouts\BerryPicker\src\sensorprocessing\..\sensorprocessing\sp_propriotuned_cnn.py:78: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  self

In [6]:
# Run the sensor processing package to process the images of a demonstration
task = exp["proprioception_testing_task"]
demos_dir = pathlib.Path(Config()["demos"]["directory"])
task_dir = pathlib.Path(demos_dir, "demos", task)
with torch.no_grad():
    for demo_dir in task_dir.iterdir():
        if not demo_dir.is_dir():
            pass
        bcd = BCDemonstration(demo_dir, sensorprocessor=None)
        for i in range(bcd.trim_from, bcd.trim_to):
            sensor_readings, _ = bcd.get_image(i)
            sensor_readings = sensor_readings.to(device)
            z = sp.process(sensor_readings)
            break

print(z)
print(z.shape)

Cameras found: ['dev2']
There are 437 steps in this demonstration
This demonstration was recorded by the following cameras: ['dev2']
sensor readings shape torch.Size([1, 3, 256, 256])
[-2.9731896  -1.9003732  -4.930805    2.060348   -1.4342488  -1.7257458
 -0.1114385   2.1723168  -4.742881   -5.831705   -1.4768347  -6.2447257
 -1.5481517   3.8437572  -6.5623317  -5.236012    1.5520467   1.9094057
  3.0569563  -4.4211493  -0.9991176  -2.9234872  -0.87173915 -0.4043593
 -3.8335443  -2.170678   -2.4310002   2.9685373  -1.4171333   3.767417
 -0.7321997  -3.4720159  -0.76205915  5.439384   -4.5618114   1.3335418
 -3.2448993  -2.4954402  -0.29288012 -0.60725766 -0.1083415  -2.6737127
 -4.5848293  -4.1542573   0.49052358 -0.64144444 -1.7204938  -3.028482
 -2.5272453  -2.232398   -0.05623122 -3.1219704  -4.1446085   1.3693383
  3.8048067  -2.0926747   1.9654616   0.68498003  0.5559831   1.6852123
 -0.12765948 -1.663783    1.7601906  -2.5111487  -2.0891857   0.21188208
  4.3100896   3.6563284  

In [7]:
# Run the sensor processing inside the BCDemonstration
task = exp["proprioception_testing_task"]
demos_dir = pathlib.Path(Config()["demos"]["directory"])
task_dir = pathlib.Path(demos_dir, "demos", task)
with torch.no_grad():
    for demo_dir in task_dir.iterdir():
        if not demo_dir.is_dir():
            pass
        bcd = BCDemonstration(demo_dir, sensorprocessor=sp)
        for i in range(bcd.trim_from, bcd.trim_to):
            #sensor_readings, _ = bcd.get_image(i)
            #sensor_readings = sensor_readings.to(device)
            #z = sp.process(sensor_readings)
            z = bcd.get_z(i)
            break

print(z)
print(z.shape)

Cameras found: ['dev2']
There are 437 steps in this demonstration
This demonstration was recorded by the following cameras: ['dev2']
sensor readings shape torch.Size([1, 3, 256, 256])
[-2.9731896  -1.9003732  -4.930805    2.060348   -1.4342488  -1.7257458
 -0.1114385   2.1723168  -4.742881   -5.831705   -1.4768347  -6.2447257
 -1.5481517   3.8437572  -6.5623317  -5.236012    1.5520467   1.9094057
  3.0569563  -4.4211493  -0.9991176  -2.9234872  -0.87173915 -0.4043593
 -3.8335443  -2.170678   -2.4310002   2.9685373  -1.4171333   3.767417
 -0.7321997  -3.4720159  -0.76205915  5.439384   -4.5618114   1.3335418
 -3.2448993  -2.4954402  -0.29288012 -0.60725766 -0.1083415  -2.6737127
 -4.5848293  -4.1542573   0.49052358 -0.64144444 -1.7204938  -3.028482
 -2.5272453  -2.232398   -0.05623122 -3.1219704  -4.1446085   1.3693383
  3.8048067  -2.0926747   1.9654616   0.68498003  0.5559831   1.6852123
 -0.12765948 -1.663783    1.7601906  -2.5111487  -2.0891857   0.21188208
  4.3100896   3.6563284  